In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
import numpy as np

In [2]:
data = pd.read_csv('data.csv')
d1 = data 
d1 = d1.drop("Origin_Airport",axis = 1)
d1 = d1.drop("Origin_City",axis = 1)
d1 = d1.drop("Destination_Airport",axis = 1)
d1 = d1.drop("Delay_Time",axis = 1)
d1 = d1.drop("Cancelled",axis = 1)
d1.head()

,Unnamed: 0,Date,Airline,Destination_City,Flight_Status
0,0,2019-12-01,YV,"Atlanta, GA",Normal
1,1,2019-12-01,YV,"Columbus, OH",Normal
2,2,2019-12-01,YV,"Harlingen/San Benito, TX",Normal
3,3,2019-12-01,YV,"Des Moines, IA",Delayed
4,4,2019-12-01,YV,"Charlotte, NC",Normal


In [3]:
date = d1['Date']
d1['Date'] = pd.to_datetime(d1['Date'], errors='coerce')
d1['MONTH'] = d1['Date'].dt.month

In [4]:
New = [d1['MONTH'],d1['Airline'],d1['Destination_City'],d1['Flight_Status']]
New_DF= pd.DataFrame(New)
New_DF = New_DF.transpose()
New_DF.head()

,MONTH,Airline,Destination_City,Flight_Status
0,12,YV,"Atlanta, GA",Normal
1,12,YV,"Columbus, OH",Normal
2,12,YV,"Harlingen/San Benito, TX",Normal
3,12,YV,"Des Moines, IA",Delayed
4,12,YV,"Charlotte, NC",Normal


In [5]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder_FS = LabelEncoder()
label_encoder_FS.fit(New_DF["Flight_Status"])
encoded_y_FS = label_encoder_FS.transform(New_DF["Flight_Status"])

label_encoder_DC = LabelEncoder()
label_encoder_DC.fit(New_DF["Destination_City"])
encoded_y_DC = label_encoder_DC.transform(New_DF["Destination_City"])

label_encoder_AL = LabelEncoder()
label_encoder_AL.fit(New_DF["Airline"])
encoded_y_AL = label_encoder_AL.transform(New_DF["Airline"])

In [6]:
Final1_Df = [New_DF['MONTH'],encoded_y_FS,encoded_y_DC,encoded_y_AL]
Final_Df = pd.DataFrame(Final1_Df)
Final_Df = Final_Df.transpose()
Final_Df = Final_Df.rename(columns = {'Unnamed 0' : 'Status','Unnamed 1': 'Dest_City','Unnamed 2': 'Airline' })
X = Final_Df.drop("Status", axis=1)
y = Final_Df["Status"]
Final_Df.head()

,MONTH,Status,Dest_City,Airline
0,12,2,6,12
1,12,2,25,12
2,12,2,43,12
3,12,1,30,12
4,12,2,18,12


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train = X_train.values
X_test = X_test.values

In [8]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train_transform = X_scaler.transform(X_train)
X_test_transform = X_scaler.transform(X_test)

In [9]:
from tensorflow.keras.utils import to_categorical
# One-hot encoding
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_test_cat.shape

(22767, 3)

In [10]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [11]:
from tensorflow.keras.layers import Dense
# Add the first layer where the input dimensions are the 561 columns of the training data
model.add(Dense(100, activation='relu', input_dim=X_train_transform.shape[1]))

In [12]:
y_train_cat.shape

(68301, 3)

In [13]:
# Add output layer
model.add(Dense(y_train_cat.shape[1], activation="softmax"))

In [14]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [16]:
 # Use the training data to fit (train) the model
model.fit(
    X_train_transform,
    y_train_cat,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
2135/2135 - 5s - loss: 0.6883 - accuracy: 0.6622
Epoch 2/100
2135/2135 - 4s - loss: 0.6775 - accuracy: 0.6639
Epoch 3/100
2135/2135 - 4s - loss: 0.6757 - accuracy: 0.6636
Epoch 4/100
2135/2135 - 4s - loss: 0.6752 - accuracy: 0.6634
Epoch 5/100
2135/2135 - 4s - loss: 0.6744 - accuracy: 0.6634
Epoch 6/100
2135/2135 - 4s - loss: 0.6740 - accuracy: 0.6634
Epoch 7/100
2135/2135 - 4s - loss: 0.6736 - accuracy: 0.6637
Epoch 8/100
2135/2135 - 5s - loss: 0.6732 - accuracy: 0.6636
Epoch 9/100
2135/2135 - 4s - loss: 0.6731 - accuracy: 0.6637
Epoch 10/100
2135/2135 - 6s - loss: 0.6730 - accuracy: 0.6637
Epoch 11/100
2135/2135 - 5s - loss: 0.6727 - accuracy: 0.6639
Epoch 12/100
2135/2135 - 4s - loss: 0.6726 - accuracy: 0.6638
Epoch 13/100
2135/2135 - 4s - loss: 0.6724 - accuracy: 0.6638
Epoch 14/100
2135/2135 - 4s - loss: 0.6724 - accuracy: 0.6641
Epoch 15/100
2135/2135 - 4s - loss: 0.6722 - accuracy: 0.6639
Epoch 16/100
2135/2135 - 4s - loss: 0.6722 - accuracy: 0.6636
Epoch 17/100
2135

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_transform, y_test_cat, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

712/712 - 1s - loss: 0.6706 - accuracy: 0.6667
Loss: 0.670600175857544, Accuracy: 0.6666666865348816


In [52]:
test = np.expand_dims(X_test_transform[492], axis=0)
print(f"Predicted class: {model.predict_classes(test)}")

Predicted class: [2]


In [51]:
Final_Df.sample(50)

,MONTH,Status,Dest_City,Airline
30447,6,2,6,4
13372,12,1,49,13
6698,12,2,69,10
32545,6,2,81,12
54062,7,1,52,11
78881,5,1,23,10
62624,10,2,94,12
13866,12,1,6,5
50315,7,1,55,5
5143,12,2,98,11


In [50]:
len(X_test_transform)

22767